In [1]:
#Lession 1 - 서울시 응답소 사이트에 접속 후 메뉴 클릭
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import math
import pandas as pd  
import openpyxl
import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 크롤링에 필요한 정보를 입력 받습니다.
print("=" *100)
print("연습문제: 서울시 응답소 게시판 크롤링하기")
print("=" *100)

query_txt = '서울시응답소'
cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir=='' :
    f_dir ='c:\\py_temp\\'
    
print()

# 저장될 위치와 이름을 지정합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬드라이버 설정 -> 응답소 사이트 접속 -> 메뉴클릭
s_time = time.time( )  # 시작시간 스탑워치 클릭

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get('https://eungdapso.seoul.go.kr/')
driver.maximize_window()

#페이지가 다 열릴 때까지 2-5초 사이 기다리기
time.sleep(random.randrange(2,5))  

#시장에게 바랍니다 메뉴 클릭하기
driver.find_element(By.XPATH,'//*[@id="gnb"]/div[4]/div[2]/div/a').click()

#Lesson 2 - 게시물 상세 정보 수집하기
# Step 4.게시물의 정보 수집

no2=[]             # 게시글 번호 컬럼
title2=[]          # 게시글 제목 컬럼
odate2=[]          # 게시글 공개일 컬럼
contents2=[]       # 게시글 상담내역 컬럼

no = 1    # 게시글 번호 카운트 변수

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 각 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(2,12) :
        
        if no > cnt :
            break
        else :
           
            #txt 형식으로 저장할 파일 지정
            f = open(ff_name, 'a',encoding='UTF-8')           

            # 게시글 제목 클릭
            print('===== %s번째 게시물의 상세내역을 추출합니다=====' %no)
            driver.find_element(By.XPATH,'//*[@id="content_cont"]/div[2]/div/form/div[4]/table/tbody/tr[%s]/td[2]/a' %i).click( )
            time.sleep(1)
                               
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
           
            # 1.번호 입력
            no2.append(no)
            
            # 2.제목 추출
            title_1 = soup.find('table').find('tbody').find_all('tr')
            title = title_1[0].get_text().replace('제목','').replace('\n','')
            print('1.제목:' , title)
            f.write('1.제목:' + title + '\n' )
            title2.append(title)
            
            # 3.공개일 추출
            odate = title_1[1].get_text().replace('공개일','').replace('\n','')
            print('2.공개일:', odate)
            f.write('2.공개일:' + odate + '\n' )
            odate2.append(odate)
                       
            # 4.상담내역 추출
            contents = soup.find('div','table_inner_desc').get_text().replace('Q. 상담내용','').replace('\n','').strip()
            print('3.상담내역:', contents , '\n')
            f.write('3.상담내역:' + contents +'\n')
            contents2.append(contents)
            f.close( )
            

            no += 1

            #이전 페이지로 돌아가기
            driver.back()
        
    time.sleep(1)      
    
    x += 1    
    try :
        driver.find_element(By.LINK_TEXT,'%s' %x).click() # 다음 페이지번호 클릭
    except :
        driver.find_element(By.LINK_TEXT,'>').click()

#Lesson 3 - 수집된 정보를 다양한 형식의 파일로 저장하기
# Step 6. 수집된 정보를 다양한 형식의 파일로 저장하기 
seoul = pd.DataFrame()
seoul['번호'] = no2
seoul['제목'] = title2
seoul['공개일'] = odate2
seoul['상담내용']=contents2

# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name,index=False, engine='openpyxl')

#Step 7. 요약 정보 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

연습문제: 서울시 응답소 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 5
크롤링 할 총 페이지 번호:  1
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\py_temp\):c:\py_temp\

1 페이지 내용 수집 시작합니다 =======================
===== 1번째 게시물의 상세내역을 추출합니다=====
1.제목: 인라인장 주차장 사용 금지 요청
2.공개일: 2022-06-27
3.상담내역: 한강 시민공원의 유일한 인라인장을주말마다 수영장 이용객을 위해 주차장으로 쓴다는데...제3주차장과 국회둔치 주차장이 있는데 꼭 인라인장을 주차장으로 써야 하나요?놀 곳 없는 아이들주말마다 가서 신나게 달리고 노는 장소였는데..개선이 필요한 것 같습니다요.그리고차가 이렇게 많이 올 정도면 수영장 인원도 많다는 얘긴데..아직 적당한 거리두기 하고 있는거 아닌가요.. 

===== 2번째 게시물의 상세내역을 추출합니다=====
1.제목: 전기차 화재 진압 프로토콜 변경 요청
2.공개일: 2022-06-14
3.상담내역: 안녕하세요. 최근 이슈가 되고 있는 전기차 화재 발생 시 진압 프로토콜에 큰 문제가 있어서 간곡히 건의 드립니다.최근 부산에서 발생한 전기차 화재 관련 뉴스 기사를 보니 현재 전기차에 화재 발생 시 자동차 주변에 가벽을 설치하고 배터리가 모두 잠길 때까지 물을 붓는다고 설명하였는데,리튬 배터리의 경우 물을 부으면 물과 리튬이 반응하여 고열이 더 발생하기 때문에 열폭주를 더욱 심하게 발생시킬 수 있습니다.전기차 화재 발생시에는 리튬 전용 소화기 사용을 해야 하고 불가능하면 최소한 물 대신 모래를 부어야 물 보다 빨리 진압이 가능 합니다.하루라도 빨리 전기차 화재 발생 시 화재 진압 프로토콜이 효율적으로 적용되어 한 명의 목숨이라도 더 살릴 수 있길 바랍니다. 

===== 3번째 게시물의 상세내역을 추출합니다=====
1.제목: Green seoul
2.공개일: 2022-06-13
3.상담내역: 이제는 도로를 넓이는것보다는  가로수

In [2]:
title_1

[<tr>
 <th scope="row">제목</th>
 <td>시장님 구로창업허브 신설에 청년제조업을 위해 힘써주세요</td>
 </tr>,
 <tr>
 <th scope="row">공개일</th>
 <td>2022-06-11</td>
 </tr>]